In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [2]:
# Load the trained model
model=load_model('model.h5')

# load scaled, onehotncoding and label encoder pickle file
with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)

with open('onehot_encoder_geo.pkl', 'rb') as file:
    onehot_encoder_geo=pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)


In [3]:
# Example iput data
input_data={
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age':40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard':1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [14]:
input_data.keys()

dict_keys(['CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'])

In [ ]:
# Transform the data using preprocessor objects
# First encode categorical variables into numericals. Then apply standard scaler 
geo_encoded=onehot_encoder_geo.transform([[input_data['Geography']]])
geo_encoded

c:\Users\tejus\OneDrive\Desktop\Extra Projects\Churn Prediction Using ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 1 stored elements and shape (1, 3)>

In [ ]:
# Get the one hot encoded values
geo_encoded.toarray()

array([[1., 0., 0.]])

In [40]:
# Get the one hot encoded features
onehot_encoder_geo.get_feature_names_out(["Geography"])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [39]:
# Create datframe which only contains one hot encoded features
geo_encoded_df=pd.DataFrame(geo_encoded.toarray(), columns=onehot_encoder_geo.get_feature_names_out(["Geography"]))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [23]:
# We cannot have only scaler values in dictionary. Converted to list of dictionary
input_df=pd.DataFrame([input_data])

In [29]:
# Concatenate the 2 dataframes
df=pd.concat([input_df, geo_encoded_df], axis=1)
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,France,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [30]:
# Drop the  Geography column
df=df.drop('Geography', axis=1)
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [32]:

# Label encode the gender
df['Gender']=label_encoder_gender.transform(df['Gender'])
df


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [34]:
# Scale the input
scaled_input=scaler.transform(df)

In [37]:
# get the prediction in the form of probability
prediction=model.predict(scaled_input)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step


array([[0.06876544]], dtype=float32)

In [38]:
prediction_proba=prediction[0][0]

# If prediction>0.5, then classify as 1, else 0
if prediction_proba>0.5:
    print('The customer is likley to churn.')
else:
    print('The customer is not likley to churn.')

The customer is not likley to churn.
